In [2]:
from dotenv import load_dotenv
from glob import glob
from pprint import pprint #json 형식 출력에 편리
import os

In [3]:
load_dotenv()

True

## Chroma DB에 직접 데이터 넣기

In [3]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name = "BAAI/bge-m3")

c:\Users\USER\AppData\Local\pypoetry\Cache\virtualenvs\langchain-env-aZmMjlRy-py3.11\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\USER\AppData\Local\pypoetry\Cache\virtualenvs\langchain-env-aZmMjlRy-py3.11\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--BAAI--bge-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlin

### DB 만들기

In [4]:
from langchain_chroma import Chroma

chroma_db = Chroma(
    collection_name="ai_sample_collection",
    persist_directory='./chroma_db',
    embedding_function=embeddings_model,
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [5]:
chroma_db.get()

Failed to send telemetry event CollectionGetEvent: capture() takes 1 positional argument but 3 were given


{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'data': None,
 'metadatas': [],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

### DB에 값 넣기

In [6]:
from langchain_core.documents import Document

documents = [
    "인공지능은 컴퓨터 과학의 한 분야입니다.",
    "머신러닝은 인공지능의 하위 분야입니다.",
    "딥러닝은 머신러닝의 한 종류입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.",
    "컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다."
]

doc_objects = []

for i, content in enumerate(documents, start=1):
    doc = Document(
        page_content = content,
        metadata = {"source" : "AI_textbook", "chapter" : f"Chapter {i}"},
    )

    doc_objects.append(doc)

doc_ids = [f"doc_{i}" for i in range(1, len(doc_objects)+1)]

added_docs_ids = chroma_db.add_documents(documents=doc_objects, ids=doc_ids)

print(f"{len(added_docs_ids)} 개의 문서가 성공적으로 DB에 추가되었습니다.")

5 개의 문서가 성공적으로 DB에 추가되었습니다.


In [8]:
query = "인공지능과 머신러닝의 관계는?"

results = chroma_db.similarity_search(query=query, k=2)

print(f"질문 : {query}")
for doc in results:
    print(f"문서 ID : {doc.metadata['chapter']}, 문서내용 : {doc.page_content}")


질문 : 인공지능과 머신러닝의 관계는?
문서 ID : Chapter 2, 문서내용 : 머신러닝은 인공지능의 하위 분야입니다.
문서 ID : Chapter 3, 문서내용 : 딥러닝은 머신러닝의 한 종류입니다.


In [9]:
chroma_db.get()

{'ids': ['doc_1', 'doc_2', 'doc_3', 'doc_4', 'doc_5'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 한 분야입니다.',
  '머신러닝은 인공지능의 하위 분야입니다.',
  '딥러닝은 머신러닝의 한 종류입니다.',
  '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.',
  '컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'chapter': 'Chapter 1', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 2', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 3', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 4', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 5', 'source': 'AI_textbook'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

### DB 에 있는 값 업데이트, 수정하기

In [ ]:
update_document_1 = Document(
    page_content="인공지능은 컴퓨터 과학의 한 분야로, 머신러닝과 딥러닝을 포함한다.",
    metadata = {"source" : "AI_textbook", "chapter" : "Chapter 6"}
)

chroma_db.update_document(document_id="doc_1", document=update_document_1) # document_id 는 ids 에 있는 값을 사용한다.

chroma_db.get()

Failed to send telemetry event CollectionUpdateEvent: capture() takes 1 positional argument but 3 were given


{'ids': ['doc_1', 'doc_2', 'doc_3', 'doc_4', 'doc_5'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 한 분야로, 머신러닝과 딥러닝을 포함한다.',
  '머신러닝은 인공지능의 하위 분야입니다.',
  '딥러닝은 머신러닝의 한 종류입니다.',
  '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.',
  '컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'chapter': 'Chapter 6', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 2', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 3', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 4', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 5', 'source': 'AI_textbook'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

### DB에 있는 값 삭제하기

In [11]:
chroma_db.delete(ids=["doc_5"])

chroma_db.get()

Failed to send telemetry event CollectionDeleteEvent: capture() takes 1 positional argument but 3 were given


{'ids': ['doc_1', 'doc_2', 'doc_3', 'doc_4'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 한 분야로, 머신러닝과 딥러닝을 포함한다.',
  '머신러닝은 인공지능의 하위 분야입니다.',
  '딥러닝은 머신러닝의 한 종류입니다.',
  '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'chapter': 'Chapter 6', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 2', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 3', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 4', 'source': 'AI_textbook'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

## 과정 총정리

### 문서 로딩

In [ ]:
text_files_path = glob(os.path.join('data','*_kr.txt'))

from langchain_community.document_loaders import TextLoader

docs = []

for text_file_path in text_files_path:
    loader = TextLoader(text_file_path, encoding='utf-8')
    docs += loader.load() # load를 하면 리스트로 반환된다. 그 안은 Document 객체가 존재

In [5]:
print(docs)

[Document(metadata={'source': 'data\\samsung_kr.txt'}, page_content='삼성전자는 1969년 설립된 세계적인 종합 전자기업으로, 1983년 반도체 사업에 본격 진출하며 메모리 반도체 중심의 글로벌 경쟁력을 쌓아왔습니다. 본사는 대한민국 경기도 수원에 있으며, 반도체 생산라인은 기흥, 화성, 평택, 그리고 미국 오스틴 및 텍사스 테일러시 등 국내외에 걸쳐 있습니다.\n\n삼성전자의 반도체 부문은 크게 메모리 사업부와 시스템LSI 사업부로 구성되어 있습니다. 메모리 부문에서는 D램, 낸드플래시 등에서 수년간 세계 시장 점유율 1위를 유지하고 있으며, AI, 클라우드, 스마트폰 등 다양한 산업에 고성능 메모리 솔루션을 공급합니다. 특히 5세대 HBM3E, 1b 나노급 D램, V-NAND 9세대 기술 등에서 혁신을 주도하고 있습니다.\n\n시스템LSI 사업부는 SoC(System-on-Chip), 이미지 센서, 디스플레이 드라이버 IC 등 비메모리 분야를 담당하며, 모바일 애플리케이션 프로세서(Exynos)와 세계 1위 점유율을 가진 이미지 센서(ISOCELL)로 경쟁력을 강화하고 있습니다. 또한, 삼성 파운드리는 3nm GAA(Gate-All-Around) 공정 양산을 세계 최초로 시작하며 TSMC와 경쟁하고 있습니다.\n\n삼성전자 파운드리 사업의 시장 점유율이 지속적으로 하락하고 있습니다. 2025년 1분기 기준, 삼성전자의 파운드리 시장 점유율은 17.7%로, 전년 동기 대비 감소했습니다. 반면, TSMC는 시장 점유율을 확대하며 67.6%를 기록하여 격차가 더욱 커지고 있는 상황입니다. '), Document(metadata={'source': 'data\\skHynix_kr.txt'}, page_content='SK하이닉스는 1983년 현대전자산업으로 출발해, 2012년 SK그룹에 인수되면서 현재의 이름을 갖게 된 대한민국 대표 반도체 제조기업입니다. 본사는 경기도 이천에 위치하고 있으며, 글로벌 반도체 

### 문서 Chunk로 나누기

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 40,
    length_function = len,
    separators = ["\n\n", "\n"],
)

chunked_docs = text_splitter.split_documents(docs)

### 문서 임베딩

In [7]:
# 허깅페이스 임베딩 모델 사용

from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name = "BAAI/bge-m3")

# embedded_docs = embedding_model.embed_documents(chunked_docs)

c:\Users\USER\AppData\Local\pypoetry\Cache\virtualenvs\langchain-env-aZmMjlRy-py3.11\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


### Vector DB로 임베딩된 문서 저장하기

#### 1. DB 만들기

In [8]:
from langchain_chroma import Chroma

chroma_db = Chroma(
    collection_name="langchain_assignment_4",
    persist_directory='./chroma_db',
    embedding_function=embedding_model,
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


#### 2. DB에 데이터 넣기

In [9]:
from langchain_core.documents import Document

docs_input = []

for i, content in enumerate(chunked_docs, start=1):
    doc = Document(
        page_content=content.page_content,
        metadata = {"chunk_number" : f"chunk {i}"}
    )

    docs_input.append(doc)

docs_ids = [f"doc_{i}" for i in range(1, len(docs_input)+1)]

chroma_db.add_documents(documents=docs_input, ids = docs_ids)

['doc_1',
 'doc_2',
 'doc_3',
 'doc_4',
 'doc_5',
 'doc_6',
 'doc_7',
 'doc_8',
 'doc_9',
 'doc_10',
 'doc_11',
 'doc_12',
 'doc_13']

In [10]:
chroma_db.get()

Failed to send telemetry event CollectionGetEvent: capture() takes 1 positional argument but 3 were given


{'ids': ['doc_1',
  'doc_2',
  'doc_3',
  'doc_4',
  'doc_5',
  'doc_6',
  'doc_7',
  'doc_8',
  'doc_9',
  'doc_10',
  'doc_11',
  'doc_12',
  'doc_13'],
 'embeddings': None,
 'documents': ['삼성전자는 1969년 설립된 세계적인 종합 전자기업으로, 1983년 반도체 사업에 본격 진출하며 메모리 반도체 중심의 글로벌 경쟁력을 쌓아왔습니다. 본사는 대한민국 경기도 수원에 있으며, 반도체 생산라인은 기흥, 화성, 평택, 그리고 미국 오스틴 및 텍사스 테일러시 등 국내외에 걸쳐 있습니다.',
  '\n삼성전자의 반도체 부문은 크게 메모리 사업부와 시스템LSI 사업부로 구성되어 있습니다. 메모리 부문에서는 D램, 낸드플래시 등에서 수년간 세계 시장 점유율 1위를 유지하고 있으며, AI, 클라우드, 스마트폰 등 다양한 산업에 고성능 메모리 솔루션을 공급합니다. 특히 5세대 HBM3E, 1b 나노급 D램, V-NAND 9세대 기술 등에서 혁신을 주도하고 있습니다.',
  '\n시스템LSI 사업부는 SoC(System-on-Chip), 이미지 센서, 디스플레이 드라이버 IC 등 비메모리 분야를 담당하며, 모바일 애플리케이션 프로세서(Exynos)와 세계 1위 점유율을 가진 이미지 센서(ISOCELL)로 경쟁력을 강화하고 있습니다. 또한, 삼성 파운드리는 3nm GAA(Gate-All-Around) 공정 양산을 세계 최초로 시작하며 TSMC와 경쟁하고 있습니다.',
  '삼성전자 파운드리 사업의 시장 점유율이 지속적으로 하락하고 있습니다. 2025년 1분기 기준, 삼성전자의 파운드리 시장 점유율은 17.7%로, 전년 동기 대비 감소했습니다. 반면, TSMC는 시장 점유율을 확대하며 67.6%를 기록하여 격차가 더욱 커지고 있는 상황입니다.',
  'SK하이닉스는 1983년 현대전자산업으로 출발해, 2012년 SK그룹에 인수되면

### DB에서 query 와 유사도 검색

In [11]:
query = "삼성은 어떤 회사인가?"

results = chroma_db.similarity_search(query=query, k=2)

print(f"질문 : {query}")
for doc in results:
    print(f"문서 ID : {doc.metadata['chunk_number']}, 문서내용 : {doc.page_content}")


Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


질문 : 삼성은 어떤 회사인가?
문서 ID : chunk 1, 문서내용 : 삼성전자는 1969년 설립된 세계적인 종합 전자기업으로, 1983년 반도체 사업에 본격 진출하며 메모리 반도체 중심의 글로벌 경쟁력을 쌓아왔습니다. 본사는 대한민국 경기도 수원에 있으며, 반도체 생산라인은 기흥, 화성, 평택, 그리고 미국 오스틴 및 텍사스 테일러시 등 국내외에 걸쳐 있습니다.
문서 ID : chunk 5, 문서내용 : SK하이닉스는 1983년 현대전자산업으로 출발해, 2012년 SK그룹에 인수되면서 현재의 이름을 갖게 된 대한민국 대표 반도체 제조기업입니다. 본사는 경기도 이천에 위치하고 있으며, 글로벌 반도체 시장에서 삼성전자에 이어 세계 2위 D램 제조업체로 자리잡고 있습니다.


## 교수님 코드

In [4]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

# 문서로딩

def load_text_files(text_files):
    data=[]
    for file in text_files:
        loader = TextLoader(file, encoding='utf-8')
        data += loader.load() #load의 반환형은 리스트이기에 append가 아닌 += 복합 연산자가 사용된다.

    return data

korean_text_files = glob(os.path.join('data',"*_kr.txt")) 
korean_data = load_text_files(korean_text_files)

In [5]:
# 문서 chunk
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer, # 이걸 넣으면 어떤 변화가 있는가, 토큰으로 청킹을 함. -> BAAI/bge-m3 모델에 맞춤형으로 토큰화 함.
    separator=r"[.!?]\s+",
    chunk_size=100,
    chunk_overlap=0,
    is_separator_regex=True,
    keep_separator=True,
)

korean_docs = text_splitter.split_documents(korean_data)

print(f"한국어 문서 수 : {len(korean_docs)}")

c:\Users\USER\AppData\Local\pypoetry\Cache\virtualenvs\langchain-env-aZmMjlRy-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


한국어 문서 수 : 19


In [6]:
# 문서 임베딩

from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name = "BAAI/bge-m3")

이건 DB 저장하는 것

In [7]:
from langchain_chroma import Chroma

# 벡터 DB
chroma_db = Chroma.from_documents(
    documents = korean_docs, # Document 리스트 형태로 되어야 함.
    embedding=embedding_model,
    collection_name="langchain_assignment04_prof",
    persist_directory='./chroma_db',
    collection_metadata={'hnsw:space' : 'cosine'}, #l2, cosine, ip, 유사도 계산을 어떻게 할 것인가.
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


이건 DB 읽어오는 것

In [4]:
from langchain_chroma import Chroma

chroma_db = Chroma(
    embedding_function=embedding_model,
    collection_name="langchain_assignment04_prof",
    persist_directory='./chroma_db',
    collection_metadata={'hnsw:space' : 'cosine'},
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [8]:
chroma_db.get()

Failed to send telemetry event CollectionGetEvent: capture() takes 1 positional argument but 3 were given


{'ids': ['8ea29d4e-9f3d-4c3e-b9ee-2ea032792806',
  '3c5758e6-74c7-4135-924a-c5df3b79efaa',
  '7392df3c-906c-4e7b-aee8-e9518025edee',
  'dde18b41-1de0-4e70-8516-fddcc49b0d9a',
  'ed53245b-675e-4f79-9e74-1d1d39e572be',
  '2ed51cd6-5941-41e3-8484-430ad679e53c',
  '1de0d562-e1d5-41de-890b-8f2b4cf9e3f9',
  '48701be5-9dc5-4b70-a7b8-29c9ca44115e',
  '0e1a68f1-a4d7-4d67-a3f2-6db3d8c8b69f',
  'f2fd6a73-f2c6-4895-9a01-11e386ddd5f8',
  'aae823a5-bdf5-4294-b516-8eb3e5035b4b',
  '498b8ab0-f551-4ad3-9e30-f1ef912287c9',
  '39b09194-a6e0-46e4-9c1e-2f00590d3860',
  '115c1dca-4bd2-45f7-ae0a-6427ba00bb62',
  'd1c0655c-6604-4a27-bf75-c7386e3f1fb9',
  '0353a178-e070-489c-be99-f115484f94b2',
  '10638d37-d3d0-4029-97cb-601d8ea92309',
  'bb053198-d333-4e47-8544-d4cf076a3347',
  '268c14d0-a387-438f-8a0e-754f406d4899'],
 'embeddings': None,
 'documents': ['삼성전자는 1969년 설립된 세계적인 종합 전자기업으로, 1983년 반도체 사업에 본격 진출하며 메모리 반도체 중심의 글로벌 경쟁력을 쌓아왔습니다. 본사는 대한민국 경기도 수원에 있으며, 반도체 생산라인은 기흥, 화성, 평택, 그리고 미국 오스틴 및 텍사스 테일러시 등 국내외에 걸

In [15]:
# 검색기
retriever = chroma_db.as_retriever(
    search_kwargs={'k':2},
)

query = "삼성전자는 언제 사업을 시작했나요?"
retriever_docs = retriever.invoke(query)

print(len(retriever_docs))

print(f"쿼리 : {query}")
print("검색 결과 : ")
for doc in retriever_docs:
    print(f" - {doc.page_content} \n [출처 : {doc.metadata['source']}]")
    print("="*30)

2
쿼리 : 삼성전자는 언제 사업을 시작했나요?
검색 결과 : 
 - 삼성전자는 1969년 설립된 세계적인 종합 전자기업으로, 1983년 반도체 사업에 본격 진출하며 메모리 반도체 중심의 글로벌 경쟁력을 쌓아왔습니다. 본사는 대한민국 경기도 수원에 있으며, 반도체 생산라인은 기흥, 화성, 평택, 그리고 미국 오스틴 및 텍사스 테일러시 등 국내외에 걸쳐 있습니다 
 [출처 : data\samsung_kr.txt]
 - SK하이닉스는 1983년 현대전자산업으로 출발해, 2012년 SK그룹에 인수되면서 현재의 이름을 갖게 된 대한민국 대표 반도체 제조기업입니다. 본사는 경기도 이천에 위치하고 있으며, 글로벌 반도체 시장에서 삼성전자에 이어 세계 2위 D램 제조업체로 자리잡고 있습니다 
 [출처 : data\skHynix_kr.txt]


삼성전자와 하이닉스에 대한 문서가 나옴. 유사도 높은 것 2개를 뽑으라고 했지만, 실제 높은 것이 1개 밖에 없어서, 하이닉스 내용은 유사도가 상대적으로 높아서 뽑힘.

## retreiver에 조건 걸기

In [25]:
chroma_mmr = chroma_db.as_retriever(
    search_type = "similarity_score_threshold",
    search_kwargs = {
        "k" : 2,
        "score_threshold" : 0.5, #유사도 임계값 결정
    }
)

In [26]:
from langchain_community.utils.math import cosine_similarity

query = "삼성전자는 언제 사업을 시작했나요?"
retriever_docs = chroma_mmr.invoke(query)

print(f"찾은 문서 갯수 : {len(retriever_docs)}")

print(f"쿼리 : {query}")
print("검색 결과 : ")
for doc in retriever_docs:
    score = cosine_similarity(
        [embedding_model.embed_query(query)],
        [embedding_model.embed_query(doc.page_content)]
    )[0][0]
    print(f" - {doc.page_content} \n [유사도 : {score:.4f}]")
    print("="*30)

찾은 문서 갯수 : 2
쿼리 : 삼성전자는 언제 사업을 시작했나요?
검색 결과 : 
 - 삼성전자는 1969년 설립된 세계적인 종합 전자기업으로, 1983년 반도체 사업에 본격 진출하며 메모리 반도체 중심의 글로벌 경쟁력을 쌓아왔습니다. 본사는 대한민국 경기도 수원에 있으며, 반도체 생산라인은 기흥, 화성, 평택, 그리고 미국 오스틴 및 텍사스 테일러시 등 국내외에 걸쳐 있습니다 
 [유사도 : 0.7325]
 - SK하이닉스는 1983년 현대전자산업으로 출발해, 2012년 SK그룹에 인수되면서 현재의 이름을 갖게 된 대한민국 대표 반도체 제조기업입니다. 본사는 경기도 이천에 위치하고 있으며, 글로벌 반도체 시장에서 삼성전자에 이어 세계 2위 D램 제조업체로 자리잡고 있습니다 
 [유사도 : 0.5688]


In [ ]:
chroma_mmr = chroma_db.as_retriever(
    search_kwargs = {
        "k" : 2,
        "where_document" : {"$contains" : '삼성'}, # document 안에 '삼성'이라는 단어가 있을 때 만 사용
        "filter": {"source": "data\\samsung_kr.txt"}, # 이 source에서 불러온 docs 만 검색하기, metadata에 있는 내용 기반
    }
)

query = "삼성전자는 언제 사업을 시작했나요?"
retriever_docs = chroma_mmr.invoke(query)

print(f"찾은 문서 갯수 : {len(retriever_docs)}")

print(f"쿼리 : {query}")
print("검색 결과 : ")
for doc in retriever_docs:
    score = cosine_similarity(
        [embedding_model.embed_query(query)],
        [embedding_model.embed_query(doc.page_content)]
    )[0][0]
    print(f" - {doc.page_content} \n [유사도 : {score:.4f}]")
    print("="*30)

찾은 문서 갯수 : 2
쿼리 : 삼성전자는 언제 사업을 시작했나요?
검색 결과 : 
 - 삼성전자는 1969년 설립된 세계적인 종합 전자기업으로, 1983년 반도체 사업에 본격 진출하며 메모리 반도체 중심의 글로벌 경쟁력을 쌓아왔습니다. 본사는 대한민국 경기도 수원에 있으며, 반도체 생산라인은 기흥, 화성, 평택, 그리고 미국 오스틴 및 텍사스 테일러시 등 국내외에 걸쳐 있습니다 
 [유사도 : 0.7325]
 - .

삼성전자의 반도체 부문은 크게 메모리 사업부와 시스템LSI 사업부로 구성되어 있습니다. 메모리 부문에서는 D램, 낸드플래시 등에서 수년간 세계 시장 점유율 1위를 유지하고 있으며, AI, 클라우드, 스마트폰 등 다양한 산업에 고성능 메모리 솔루션을 공급합니다 
 [유사도 : 0.5510]


## 챗 봇 만들기

In [29]:
retriever = chroma_db.as_retriever(
    search_kwargs={"k":2},
)

In [30]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

template = """Answer the question based only on the following context.
Do not use any external information or knowledge. 
If the answer is not in the context, answer "잘 모르겠습니다.".

[Context]
{context}

[Question] 
{question}

[Answer]
"""

prompt = ChatPromptTemplate.from_template(template=template)

def format_docs(documents):
    return "\n\n".join([doc.page_content for doc in documents])

llm = ChatOpenAI(
    model= "gpt-4o-mini",
    temperature = 0,
)

In [31]:
retriever_chain = retriever | format_docs

In [32]:
rag_chain = (
    {"context" : retriever_chain, "question" : RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

#### 이 아래 챗봇의 흐름

메세지와 히스토리가 있음

히스토리는 이제까지의 대화 내용을 전부 담고 있음

메세지를 통해서 rag_chain을 기반으로한 llm 답변을 먼저 받음

이제까지의 히스토리, rag_chain 기반 답변, 사용자의 질문을 기반으로
새로운 llm에 질문을 넣음(일반 llm)

히스토리와 rag기반 답변으로 질문에 답변

In [ ]:
import gradio as gr
from langchain_core.messages import HumanMessage, AIMessage
# 이걸 굳이 쓰는 이유는?

def answer_invoke(message, history):

    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))

    history_langchain_format.append(HumanMessage(content=message))

    response = rag_chain.invoke(message)

    final_answer = llm.invoke(
        history_langchain_format[:-1] + [AIMessage(content=response)] + [HumanMessage(content=message)]
    )

    return final_answer.content

demo = gr.ChatInterface(fn=answer_invoke, title="QA Bot")

demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


c:\Users\USER\AppData\Local\pypoetry\Cache\virtualenvs\langchain-env-aZmMjlRy-py3.11\Lib\site-packages\gradio\analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


현재 챗 봇은 대화 내용을 기억하지 못함. -> 기억하도록 변경하기

In [43]:
demo.close()

Closing server running on port: 7860
